In [8]:
import sys
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [9]:
def load_data(messages_filepath, categories_filepath):
    """
    Function loads two datasets i.e. messages and categories, from csv source files and then merges the datasets based on a common id column
    
    Inputs:
    messages_filepath - path for csv file containing messages dataset
    categories_filepath - path for csv file containing categories dataset
    
    Outputs:
    dataframe - merged df containing data from messages & categories dataset
    
    """
    #load messages dataset
    messages = pd.read_csv(messages_filepath)
    
    #load categories dataset
    categories = pd.read_csv(categories_filepath)
    
    #merge datasets
    df = pd.merge(messages, categories, on = "id")
    
    return df

In [10]:
path = "D:\\personal_projects\\udacity_ds_nanodegree\\disaster_response_project\\data\\"

messages_path = path+"disaster_messages.csv"
categories_path = path+"disaster_categories.csv"

In [13]:
load_data(messages_path,categories_path)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...
...,...,...,...,...,...
26381,30261,The training demonstrated how to enhance micro...,NaN,news,related-0;request-0;offer-0;aid_related-0;medi...
26382,30262,A suitable candidate has been selected and OCH...,NaN,news,related-0;request-0;offer-0;aid_related-0;medi...
26383,30263,"Proshika, operating in Cox's Bazar municipalit...",NaN,news,related-1;request-0;offer-0;aid_related-0;medi...
26384,30264,"Some 2,000 women protesting against the conduc...",NaN,news,related-1;request-0;offer-0;aid_related-1;medi...


In [23]:
def clean_data(df):
    """
    Clean df through a series of dataframe manipulation processes i.e.:
    * converting categories from string type to binary values
    * removing duplicates
    * trim out rows that are tagged with the wrong label i.e. related column should only have values 0 or 1
    
    inputs:
    dataframe - merged df containing data from messages & categories dataset
    
    outputs:
    cleaned dataframe - dataframe containing cleaned version of input dataframe
    
    """
    # create a dataframe of the 36 individual category columns
    categories = df["categories"].str.split(pat=";", expand=True)
    
    # select the first row of the categories dataframe
    row = categories.iloc[0]

    # use first row to extract a list of new column names for categories up until the second to last character
    category_colnames = row.str.slice(0,-2)
    
    # rename the columns of categories df
    categories.columns = category_colnames
    
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].str.slice(-1)
        
        # convert column from string to numeric
        categories[column] = categories[column].astype(int)

        
    # drop the original categories column from main df
    df.drop(["categories"], axis=1, inplace=True)

    # concatenate the original dataframe with the new `categories` dataframe
    df = pd.concat([df, categories], sort=False, axis=1)
    
    # drop duplicates
    df = df.drop_duplicates()
    
    # trim df to only have rows with wanted labels/tags i,e related column should only have 0 OR 1
    df = df[(df["related"] == 0)|(df["related"] == 1)]

    return df

In [24]:
bb = clean_data(load_data(messages_path,categories_path))

In [37]:
def save_data(df, database_filename):
    """
    Save data in dataframe to an sqlite database
    
    inputs:
    cleaned dataframe - dataframe containing cleaned version of merged message and 
    categories data
    database_filename -  filename for output database (string)
       
    outputs:
    sqlite db containing input dataframe data
    """
    
    engine = create_engine('sqlite:///' + database_filename)
    df.to_sql(database_filename, engine, index=False, if_exists='replace')

In [ ]:
def main():
    if len(sys.argv) == 4:

        messages_filepath, categories_filepath, database_filepath = sys.argv[1:]

        print('Loading data...\n    MESSAGES: {}\n    CATEGORIES: {}'
              .format(messages_filepath, categories_filepath))
        df = load_data(messages_filepath, categories_filepath)

        print('Cleaning data...')
        df = clean_data(df)
        
        print('Saving data...\n    DATABASE: {}'.format(database_filepath))
        save_data(df, database_filepath)
        
        print('Cleaned data saved to database!')
    
    else:
        print('Please provide the filepaths of the messages and categories '\
              'datasets as the first and second argument respectively, as '\
              'well as the filepath of the database to save the cleaned data '\
              'to as the third argument. \n\nExample: python process_data.py '\
              'disaster_messages.csv disaster_categories.csv '\
              'DisasterResponse.db')


if __name__ == '__main__':
    main()